In [1]:
import sys
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
import string
import datetime

In [2]:
#Import shared functions
sys.path.append('..\..')
from IPM_Shared_Code_public.Python.utils import get_config

In [3]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['dev_server']
sladb = config['db']['sladb']

In [4]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + sladb + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

## Write all of the sql queries required for the tests

In [5]:
sql_sla_season = 'select * from sladb.dbo.tbl_sla_season'

In [186]:
# sql_season_definition = 'select * from sladb.dbo.tbl_ref_sla_season_definition' 

In [6]:
sql_change_request = 'select * from sladb.dbo.tbl_change_request'

In [7]:
sql_change_request_status = 'select * from sladb.dbo.tbl_change_request_status'

In [8]:
sql_unit_sla_season = 'select * from sladb.dbo.tbl_unit_sla_season'

In [9]:
sql_vw_sla_pivot = 'select * from vw_sla_code_pivot'

In [10]:
sql_ref_calendar = 'select * from tbl_ref_calendar'

In [11]:
sql_day_name = 'select * from tbl_ref_sla_season_day_name'

In [157]:
sql_ref_unit = 'select * from tbl_ref_unit'

In [187]:
sql_ref_sla_code = 'select * from tbl_ref_sla_code'

In [12]:
# sql_month_tbl = 'select * from tbl_ref_sla_season_month_name'

## Execute all of the sql queries and store the results

In [177]:
before_season_definition = pd.read_sql(con = engine, sql = sql_season_definition)

In [178]:
before_season_definition

,season_date_ref_id,season_id,date_ref_fixed,month_name_desc,date_ref_day_number,day_name_desc,day_rank_id,date_type_id
0,1,1,True,January,1,None,None,1
1,2,1,True,December,31,None,None,2
2,3,2,True,May,1,None,None,1
3,4,2,True,October,1,None,None,2
4,5,3,True,March,1,None,None,1
5,6,3,True,October,31,None,None,2


In [158]:
ref_unit_tbl  = pd.read_sql(con = engine, sql = sql_ref_unit)

In [188]:
ref_unit_tbl.head()

,unit_id,unit_desc,unit_class,unit_mrc,unit_status,gisobjid,unit_commiss,unit_updated,unit_withdraw
0,-,Fordham Bedford Lot Busters,PARK,M-11,D,100004689.0,2010-01-06,2018-12-20 12:40:52.483,2010-04-01
1,11772,Fordham Bedford Lot Busters,PARK,M-11,D,NaN,2011-07-28,2018-12-20 12:40:52.810,2014-04-11
2,12654,Fordham Bedford Lot Busters,PARK,M-11,D,NaN,2012-03-06,2018-12-20 12:40:52.983,2014-04-11
3,14301,Fordham Bedford Lot Busters,PARK,M-11,D,NaN,2013-06-03,2018-12-20 12:40:53.150,2014-04-11
4,14918,ZONE,ZONE,M-11,D,100008685.0,2015-02-19,2018-12-20 12:41:07.357,2015-02-19


In [189]:
before_sla_code = pd.read_sql(con = engine, sql = sql_ref_sla_code)

In [190]:
before_sla_code

,sla_code


In [13]:
calendar_tbl  = pd.read_sql(con = engine, sql = sql_ref_calendar)

In [14]:
date_cols_calendar = ['ref_date']

In [15]:
for i in date_cols_calendar:
    calendar_tbl[i] = pd.to_datetime(calendar_tbl[i])

In [152]:
calendar_tbl.head()

,ref_date,ref_year,month_name_desc,day_number,day_name_desc,day_rank_id
0,2014-01-01,2014,January,1,Wednesday,1
1,2014-01-02,2014,January,2,Thursday,1
2,2014-01-03,2014,January,3,Friday,1
3,2014-01-04,2014,January,4,Saturday,1
4,2014-01-05,2014,January,5,Sunday,1


In [16]:
day_name_tbl  = pd.read_sql(con = engine, sql = sql_day_name)
day_name_tbl.head()

,day_name_num,day_name_desc,ndays_sunday,ndays_saturday
0,6,Friday,2,1
1,2,Monday,6,5
2,7,Saturday,1,0
3,1,Sunday,0,6
4,5,Thursday,3,2


In [17]:
before_sla_season = pd.read_sql(con = engine, sql = sql_sla_season)

In [18]:
date_cols_season = [
    'effective_start', 'effective_end', 'created_date_utc', 'updated_date_utc',
    'effective_start_adj', 'effective_end_adj'
]

In [19]:
for i in date_cols_season:
    before_sla_season[i] = pd.to_datetime(before_sla_season[i])

In [20]:
before_sla_season.head()

,season_id,season_desc,year_round,effective,effective_end,effective_start,created_date_utc,updated_date_utc,effective_start_adj,effective_end_adj
0,1,"Year-round, not seasonal",True,False,2020-07-24,2014-01-01,2020-03-03 19:12:12.710,2020-07-28 20:16:56.850,2014-01-05,2020-07-25
1,2,"Beaches, etc.",False,False,2020-07-25,2019-07-01,2020-03-03 19:12:12.723,2020-07-29 12:39:31.760,2019-07-07,2020-07-25
2,3,"Ballfields, etc.",False,False,2020-07-25,2019-07-01,2020-03-03 19:12:12.723,2020-07-29 12:39:31.760,2019-07-07,2020-07-25
3,4,"NY Pause, COVID-19 Pandemic",True,False,2020-07-04,2020-03-22,2020-04-20 21:52:46.403,2020-07-01 15:20:36.193,2020-03-22,2020-07-04
4,6,New year round test season.,True,True,NaT,2020-07-24,2020-07-24 23:41:46.543,2020-07-28 18:31:55.867,2020-07-26,NaT


In [21]:
before_change_request = pd.read_sql(con = engine, sql = sql_change_request)

In [22]:
before_change_request.head()

,change_request_id,unit_id,sla_code,season_id,effective_start,change_request_justification,effective_start_adj,sla_change_status,change_request_comments
0,108462,M299,16,1,2019-07-01,AMPS updates since original review,2019-07-07,2,None
1,108463,M300,1,1,2014-01-01,Original Review,2014-01-05,2,None
2,108464,M301,1,1,2014-01-01,Original Review,2014-01-05,2,None
3,108465,M302,6,1,2014-01-01,Original Review,2014-01-05,2,None
4,108466,M303,1,1,2014-01-01,Original Review,2014-01-05,2,None


In [23]:
date_cols_change_req = ['effective_start', 'effective_start_adj']

In [24]:
for i in date_cols_change_req:
    before_change_request[i] = pd.to_datetime(before_change_request[i])

In [25]:
before_change_request_status = pd.read_sql(con = engine, sql = sql_change_request_status)

In [26]:
before_change_request_status.head()

,change_request_status_id,change_request_id,sla_change_status,created_date_utc,created_user
0,1,114208,1,2020-03-03 04:00:00,0000000
1,2,114207,1,2020-03-03 04:00:00,0000000
2,3,114206,1,2020-03-03 04:00:00,0000000
3,4,114205,1,2020-03-03 04:00:00,0000000
4,5,114204,1,2020-03-03 04:00:00,0000000


In [27]:
date_cols_change_req_stat = ['created_date_utc']

In [28]:
for i in date_cols_change_req_stat:
    before_change_request_status[i] = pd.to_datetime(before_change_request_status[i])

In [29]:
before_unit_season = pd.read_sql(con = engine, sql = sql_unit_sla_season)
before_unit_season.head()

,sla_season_id,unit_id,sla_code,season_id,effective,effective_start,effective_end,change_request_id,created_date_utc,updated_date_utc,effective_start_adj,effective_end_adj
0,1,-,16,1,False,2019-07-01,2020-07-24,109721,NaT,NaT,2019-07-07,2020-07-25
1,2,11772,16,1,False,2019-07-01,2020-07-24,109722,NaT,NaT,2019-07-07,2020-07-25
2,3,12654,16,1,False,2019-07-01,2020-07-24,109723,NaT,NaT,2019-07-07,2020-07-25
3,4,14301,16,1,False,2019-07-01,2020-07-24,109724,NaT,NaT,2019-07-07,2020-07-25
4,5,14918,16,1,False,2019-07-01,2020-07-24,109725,NaT,NaT,2019-07-07,2020-07-25


In [30]:
date_cols_unit_season = [
    'effective_start', 'effective_end', 'created_date_utc', 'updated_date_utc',
    'effective_start_adj', 'effective_end_adj'
]

In [31]:
for i in date_cols_unit_season:
    before_unit_season[i] = pd.to_datetime(before_unit_season[i])

In [32]:
today = datetime.datetime.today()

In [33]:
timing = []

In [34]:
descriptions = []

In [35]:
results = []

## Before Tests

### First before test

In [36]:
timing.append('before')

In [37]:
descriptions.append(
    '''In tbl_unit_change_request test to make sure the 'effective_start_adj' \
is between 'effective_start_adj' and 'effective_end_adj' from tbl_sla_season \
if not null. If 'effective_end_adj' is null, replace with today's date+1year. If \
this condition is not met, 'sla_change_status' must == 4''')

In [38]:
# month_tbl

In [39]:
temp_df = before_change_request.merge(before_sla_season[[
    'season_id', 'effective_start_adj', 'effective_end_adj'
]],
                                      on='season_id')[[
                                          'unit_id', 'season_id',
                                          'effective_start_adj_x',
                                          'effective_start_adj_y',
                                          'effective_end_adj',
                                          'sla_change_status'
                                      ]].copy()
temp_df.head()

,unit_id,season_id,effective_start_adj_x,effective_start_adj_y,effective_end_adj,sla_change_status
0,M299,1,2019-07-07,2014-01-05,2020-07-25,2
1,M300,1,2014-01-05,2014-01-05,2020-07-25,2
2,M301,1,2014-01-05,2014-01-05,2020-07-25,2
3,M302,1,2014-01-05,2014-01-05,2020-07-25,2
4,M303,1,2014-01-05,2014-01-05,2020-07-25,2


In [40]:
temp_df['effective_end_adj'].fillna(datetime.datetime(
            today.year + 1, today.month, today.day), inplace=True)

In [41]:
temp_df2 = temp_df[
        (temp_df['effective_start_adj_x'] < temp_df['effective_start_adj_y'])
        | (temp_df['effective_start_adj_x'] > temp_df['effective_end_adj'])]

In [42]:
results.append(
    len(temp_df2[temp_df2['sla_change_status']!=4])
    == 0)

In [43]:
temp_df2[temp_df2['sla_change_status']!=4]

,unit_id,season_id,effective_start_adj_x,effective_start_adj_y,effective_end_adj,sla_change_status
8954,R016-01,1,2020-08-02,2014-01-05,2020-07-25,1
8955,Q458-01,1,2020-08-09,2014-01-05,2020-07-25,1


In [44]:
results

[False]

### Second before test

In [45]:
timing.append('before')

In [46]:
descriptions.append(
    '''In tbl_unit_sla_season test that 'effective_start_adj' \
is the Sunday after 'effective_start' or the Sunday after 'created_date_utc' in \
tbl_change_request_status''')

In [47]:
# calendar_tbl

In [48]:
temp_df = before_unit_season[['effective_start', 'effective_start_adj']].merge(
    calendar_tbl[['ref_date', 'day_name_desc']],
    left_on='effective_start',
    right_on='ref_date').merge(day_name_tbl[['day_name_desc', 'ndays_sunday']])
temp_df.head()

,effective_start,effective_start_adj,ref_date,day_name_desc,ndays_sunday
0,2019-07-01,2019-07-07,2019-07-01,Monday,6
1,2019-07-01,2019-07-07,2019-07-01,Monday,6
2,2019-07-01,2019-07-07,2019-07-01,Monday,6
3,2019-07-01,2019-07-07,2019-07-01,Monday,6
4,2019-07-01,2019-07-07,2019-07-01,Monday,6


In [49]:
def get_day_diff(row, col1,col2):
    return (row[col1]-row[col2]).days

In [50]:
temp_df['day_diff'] = temp_df.apply(lambda row: get_day_diff(row, 'effective_start_adj','effective_start'),axis=1)

In [51]:
# before_change_request_status.head()

In [52]:
temp_df2 = before_unit_season[[
    'change_request_id', 'effective_start', 'effective_start_adj'
]].merge(
    before_change_request_status[['change_request_id', 'created_date_utc']],
    on='change_request_id')
temp_df2.head()

,change_request_id,effective_start,effective_start_adj,created_date_utc
0,109721,2019-07-01,2019-07-07,2020-03-03 04:00:00
1,109721,2019-07-01,2019-07-07,2019-07-01 00:00:00
2,109722,2019-07-01,2019-07-07,2020-03-03 04:00:00
3,109722,2019-07-01,2019-07-07,2019-07-01 00:00:00
4,109723,2019-07-01,2019-07-07,2020-03-03 04:00:00


In [53]:
# temp_df2[temp_df2['created_date_utc']<temp_df2['effective_start']]

In [54]:
# before_unit_season.head()

In [55]:
# temp_df

In [56]:
# temp_df[temp_df()]

In [57]:
# before_unit_season.head()

In [58]:
# day_name_tbl

In [59]:
# temp_df = before_change_request[[
#     'change_request_id', 'effective_start', 'effective_start_adj'
# ]].merge(before_unit_season[[
#     'change_request_id', 'effective_start', 'effective_start_adj',
#     'effective_end', 'effective_end_adj', 'created_date_utc'
# ]], on = 'change_request_id')
# temp_df.head()

In [60]:
# temp_df[pd.isnull(temp_df['effective_end_adj'])]

In [61]:
# before_unit_season.head()

In [62]:
results.append(
    len(temp_df[temp_df['day_diff']!=temp_df['ndays_sunday']])==0)

### Third before test

In [63]:
timing.append('before')

In [64]:
descriptions.append('''In tbl_unit_sla_season (or tbl_change_request) test \
that each unit only has one entry with 'sla_change_status'==1''')

In [65]:
# before_change_request.head()

In [66]:
# temp_df = before_change_request.groupby('unit_id').count()
# units_mult_stats = temp_df[temp_df['sla_change_status']>1].index.values

In [67]:
# temp_df2 = before_change_request[before_change_request['unit_id'].isin(units_mult_stats)][
#     ['unit_id', 'sla_change_status']].sort_values(by='unit_id')

In [68]:
temp_df = before_change_request[before_change_request['sla_change_status'] ==
                                1].groupby('unit_id').count()

In [69]:
results.append(len(temp_df[temp_df['sla_change_status']>1])==0)

### Fourth before test

In [78]:
timing.append('before')

In [79]:
descriptions.append('''In tbl_change_request test that each entry has \
a unique combination 'unit_id', 'effective_start_adj', and 'sla_change_status' \
fields''')

In [80]:
# before_change_request.head()

In [81]:
# before_unit_season.head()

In [87]:
temp_df = before_change_request.groupby(
    ['unit_id', 'effective_start_adj', 'sla_change_status']).count()

In [90]:
# len(temp_df[temp_df['sla_code']>1])==0

In [77]:
results.append(len(temp_df[temp_df['sla_code']>1])==0)

### Fifth before test

In [78]:
timing.append('before')

In [79]:
descriptions.append('''In tbl_unit_sla_season test that each entry has \
a unique combination 'unit_id' and 'effective_start_adj' ''')

In [80]:
# before_change_request.head()

In [81]:
# before_unit_season.head()

In [91]:
temp_df = before_unit_season.groupby(
    ['unit_id', 'effective_start_adj']).count()

In [93]:
len(temp_df[temp_df['sla_code']>1])==0

True

In [94]:
results.append(len(temp_df[temp_df['sla_code']>1])==0)

### Sixth before test

In [41]:
timing.append('before')

In [42]:
descriptions.append('''In tbl_unit_sla_season test there are no overlaps \
in dates ('effective_start' and 'effective_end')''')

In [134]:
# temp_df[['sla_season_id','effective_start_adj','effective_end_adj']]

In [143]:
# temp_df = before_unit_season.groupby(
#     ['unit_id']).count()
# temp_df.head()

In [137]:
sql_gaps_1 = '''
select unit_id,
       datediff(day, min(effective_start), max(effective_end)) as daydiff
from sladb.dbo.tbl_unit_sla_season
group by unit_id
'''

sql_gaps_2 = '''select unit_id,
       case when count(*) > 1 then sum(daydiff) + (count(*) - 1)
            else sum(daydiff)
       end as daydiff
from (select unit_id,
             datediff(day, effective_start, effective_end) as daydiff
      from sladb.dbo.tbl_unit_sla_season) as t
group by unit_id
'''

In [138]:
gaps_df1 = pd.read_sql(con = engine, sql = sql_gaps_1)
gaps_df2 = pd.read_sql(con = engine, sql = sql_gaps_2)

In [139]:
temp_df = gaps_df1.merge(gaps_df2, on='unit_id')

In [140]:
temp_df['diff'] = temp_df['daydiff_y'] - temp_df['daydiff_x']

In [142]:
temp_df[temp_df['diff']==0]

,unit_id,daydiff_x,daydiff_y,diff


In [133]:
before_unit_season[before_unit_season['unit_id'].isin(
    ['X039-ZN15', 'Q458-ZN02'])].sort_values(by='unit_id')[[
        'unit_id','effective', 'effective_start', 'effective_end', 'effective_start_adj',
        'effective_end_adj'
    ]]

,unit_id,effective,effective_start,effective_end,effective_start_adj,effective_end_adj
6757,Q458-ZN02,False,2020-03-22,2020-07-04,2020-03-22,2020-07-04
18137,Q458-ZN02,False,2020-08-14,2020-08-16,2020-08-16,2020-08-22
4893,X039-ZN15,False,2014-01-01,2019-06-30,2014-01-05,2019-07-06
5766,X039-ZN15,False,2019-07-01,2020-03-21,2019-07-07,2020-03-21
6440,X039-ZN15,False,2020-03-22,2020-07-01,2020-03-22,2020-07-04
10663,X039-ZN15,False,2020-07-05,2020-07-29,2020-07-05,2020-08-01
18059,X039-ZN15,False,2020-07-26,2020-08-08,2020-07-26,2020-08-08
18127,X039-ZN15,False,2020-08-07,2020-08-15,2020-08-09,2020-08-15


### Seventh before test

In [41]:
timing.append('before')

In [42]:
descriptions.append('''In tbl_unit_sla_season test there are no overlaps \
in dates ('effective_start_adj' and 'effective_end_adj')''')

In [135]:
# temp_df[['sla_season_id','effective_start_adj','effective_end_adj']]

In [144]:
# temp_df = before_unit_season.groupby(
#     ['unit_id']).count()
# temp_df.head()

In [179]:
sql_gaps_1 = '''
select unit_id,
       datediff(day, min(effective_start_adj), max(effective_end_adj)) as daydiff
from sladb.dbo.tbl_unit_sla_season
group by unit_id
'''

sql_gaps_2 = '''select unit_id,
       case when count(*) > 1 then sum(daydiff) + (count(*) - 2)
            else sum(daydiff)
       end as daydiff
from (select unit_id,
             datediff(day, effective_start_adj, effective_end_adj) as daydiff
      from sladb.dbo.tbl_unit_sla_season) as t
group by unit_id
'''

In [180]:
gaps_df1 = pd.read_sql(con = engine, sql = sql_gaps_1)
gaps_df2 = pd.read_sql(con = engine, sql = sql_gaps_2)

In [182]:
# gaps_df1

In [183]:
temp_df = gaps_df1.merge(gaps_df2, on='unit_id')

In [184]:
temp_df['diff'] = temp_df['daydiff_y'] - temp_df['daydiff_x']

In [185]:
temp_df[temp_df['diff']!=0]

,unit_id,daydiff_x,daydiff_y,diff
3,MZ471,NaN,NaN,NaN
8,R129,NaN,NaN,NaN
9,XZ83,NaN,NaN,NaN
11,X285,NaN,NaN,NaN
12,M010-ZN01,NaN,NaN,NaN
...,...,...,...,...
4990,B425,NaN,NaN,NaN
4991,Q357F,NaN,NaN,NaN
4992,X148O-XZ404,NaN,NaN,NaN
4993,Q162G11,NaN,NaN,NaN


In [133]:
before_unit_season[before_unit_season['unit_id'].isin(
    ['X039-ZN15', 'Q458-ZN02'])].sort_values(by='unit_id')[[
        'unit_id','effective', 'effective_start', 'effective_end', 'effective_start_adj',
        'effective_end_adj'
    ]]

,unit_id,effective,effective_start,effective_end,effective_start_adj,effective_end_adj
6757,Q458-ZN02,False,2020-03-22,2020-07-04,2020-03-22,2020-07-04
18137,Q458-ZN02,False,2020-08-14,2020-08-16,2020-08-16,2020-08-22
4893,X039-ZN15,False,2014-01-01,2019-06-30,2014-01-05,2019-07-06
5766,X039-ZN15,False,2019-07-01,2020-03-21,2019-07-07,2020-03-21
6440,X039-ZN15,False,2020-03-22,2020-07-01,2020-03-22,2020-07-04
10663,X039-ZN15,False,2020-07-05,2020-07-29,2020-07-05,2020-08-01
18059,X039-ZN15,False,2020-07-26,2020-08-08,2020-07-26,2020-08-08
18127,X039-ZN15,False,2020-08-07,2020-08-15,2020-08-09,2020-08-15


In [136]:
# before_unit_season[(before_unit_season['unit_id'].isin(
#     temp_df.index.values))][[
#         'unit_id', 'sla_season_id', 'effective_start_adj', 'effective_end_adj'
#     ]].sort_values(by=['unit_id', 'effective_start_adj'])

### Eighth before test

In [45]:
timing.append('before')

In [46]:
descriptions.append(
    '''For year-round seasons, test that 'date_ref_fixed'==1,\
    and start/end dates are Jan 1/Dec 31 ''')

In [150]:
before_change_request.head()

,change_request_id,unit_id,sla_code,season_id,effective_start,change_request_justification,effective_start_adj,sla_change_status,change_request_comments
0,108462,M299,16,1,2019-07-01,AMPS updates since original review,2019-07-07,2,None
1,108463,M300,1,1,2014-01-01,Original Review,2014-01-05,2,None
2,108464,M301,1,1,2014-01-01,Original Review,2014-01-05,2,None
3,108465,M302,6,1,2014-01-01,Original Review,2014-01-05,2,None
4,108466,M303,1,1,2014-01-01,Original Review,2014-01-05,2,None


In [ ]:
befo

In [168]:
before_change_request[before_change_request['season_id'].isin(
    before_sla_season[before_sla_season['year_round'] ==
                      True].season_id.values)].merge(before_unit_season,
                                                     on='change_request_id').head()

,change_request_id,unit_id_x,sla_code_x,season_id_x,effective_start_x,change_request_justification,effective_start_adj_x,sla_change_status,change_request_comments,sla_season_id,unit_id_y,sla_code_y,season_id_y,effective,effective_start_y,effective_end,created_date_utc,updated_date_utc,effective_start_adj_y,effective_end_adj
0,108462,M299,16,1,2019-07-01,AMPS updates since original review,2019-07-07,2,None,2017,M299,16,1,False,2019-07-01,2020-07-24,NaT,NaT,2019-07-07,2020-07-25
1,108463,M300,1,1,2014-01-01,Original Review,2014-01-05,2,None,2018,M300,1,1,False,2014-01-01,2020-03-21,NaT,NaT,2014-01-05,2020-03-21
2,108464,M301,1,1,2014-01-01,Original Review,2014-01-05,2,None,2019,M301,1,1,False,2014-01-01,2020-04-04,NaT,NaT,2014-01-05,2020-04-04
3,108465,M302,6,1,2014-01-01,Original Review,2014-01-05,2,None,2020,M302,6,1,False,2014-01-01,2020-03-21,NaT,NaT,2014-01-05,2020-03-21
4,108466,M303,1,1,2014-01-01,Original Review,2014-01-05,2,None,2021,M303,1,1,False,2014-01-01,2020-04-04,NaT,NaT,2014-01-05,2020-04-04


In [163]:
before_change_request.head()

,change_request_id,unit_id,sla_code,season_id,effective_start,change_request_justification,effective_start_adj,sla_change_status,change_request_comments
0,108462,M299,16,1,2019-07-01,AMPS updates since original review,2019-07-07,2,None
1,108463,M300,1,1,2014-01-01,Original Review,2014-01-05,2,None
2,108464,M301,1,1,2014-01-01,Original Review,2014-01-05,2,None
3,108465,M302,6,1,2014-01-01,Original Review,2014-01-05,2,None
4,108466,M303,1,1,2014-01-01,Original Review,2014-01-05,2,None


In [154]:
before_unit_season.head()

,sla_season_id,unit_id,sla_code,season_id,effective,effective_start,effective_end,change_request_id,created_date_utc,updated_date_utc,effective_start_adj,effective_end_adj
0,1,-,16,1,False,2019-07-01,2020-07-24,109721,NaT,NaT,2019-07-07,2020-07-25
1,2,11772,16,1,False,2019-07-01,2020-07-24,109722,NaT,NaT,2019-07-07,2020-07-25
2,3,12654,16,1,False,2019-07-01,2020-07-24,109723,NaT,NaT,2019-07-07,2020-07-25
3,4,14301,16,1,False,2019-07-01,2020-07-24,109724,NaT,NaT,2019-07-07,2020-07-25
4,5,14918,16,1,False,2019-07-01,2020-07-24,109725,NaT,NaT,2019-07-07,2020-07-25


In [ ]:
befo

In [173]:
before_sla_season[before_sla_season['year_round']==True].merge(before_sla_season.sla_code.values

,season_id,season_desc,year_round,effective,effective_end,effective_start,created_date_utc,updated_date_utc,effective_start_adj,effective_end_adj
0,1,"Year-round, not seasonal",True,False,2020-07-24,2014-01-01,2020-03-03 19:12:12.710,2020-07-28 20:16:56.850,2014-01-05,2020-07-25
3,4,"NY Pause, COVID-19 Pandemic",True,False,2020-07-04,2020-03-22,2020-04-20 21:52:46.403,2020-07-01 15:20:36.193,2020-03-22,2020-07-04
4,6,New year round test season.,True,True,NaT,2020-07-24,2020-07-24 23:41:46.543,2020-07-28 18:31:55.867,2020-07-26,NaT
13,20,Season test 5,True,False,2020-08-15,2020-08-09,2020-08-17 18:19:08.953,2020-08-26 16:49:57.533,2020-08-09,2020-08-15
15,23,Season test 7,True,True,NaT,2020-08-16,2020-08-24 13:39:17.280,2020-08-24 20:04:47.333,2020-08-16,NaT


In [47]:
temp_df = before_season_definition.merge(
    before_sla_season[before_sla_season['year_round'] == True],
    on='season_id')[[
        'season_id', 'year_round', 'date_ref_fixed', 'month_name_desc',
        'date_ref_day_number', 'date_type_id'
    ]]
temp_df

,season_id,year_round,date_ref_fixed,month_name_desc,date_ref_day_number,date_type_id
0,1,True,True,January,1.0,1
1,1,True,True,December,31.0,2
2,4,True,True,January,1.0,1
3,4,True,True,December,31.0,2
4,6,True,True,January,1.0,1
5,6,True,True,December,31.0,2
6,20,True,True,January,1.0,1
7,20,True,True,December,31.0,2
8,23,True,True,January,1.0,1
9,23,True,True,December,31.0,2


In [48]:
len(temp_df)==len(temp_df[((temp_df['date_type_id'] == 1)
         & (temp_df['month_name_desc'] == 'January')
         & (temp_df['date_ref_day_number'] == 1)) &
        (temp_df['date_ref_fixed'] == 1) |
        ((temp_df['date_type_id'] == 2)
         & (temp_df['month_name_desc'] == 'December')
        & (temp_df['date_ref_day_number'] == 31)) &
        (temp_df['date_ref_fixed'] == 1)])

True

In [49]:
results.append(
    len(temp_df) == len(temp_df[((temp_df['date_type_id'] == 1)
                                 & (temp_df['month_name_desc'] == 'January')
                                 & (temp_df['date_ref_day_number'] == 1))
                                & (temp_df['date_ref_fixed'] == 1)
                                | ((temp_df['date_type_id'] == 2)
                                   & (temp_df['month_name_desc'] == 'December')
                                   & (temp_df['date_ref_day_number'] == 31))
                                & (temp_df['date_ref_fixed'] == 1)]))

### Ninth before test

In [50]:
timing.append('before')

In [51]:
# before_season_definition

In [52]:
descriptions.append(
    '''For periodic seasons, test that the difference between start/end months is\
    between 2 and 10 inclusive ''')

In [160]:
ref_unit_tbl.head()

,unit_id,unit_desc,unit_class,unit_mrc,unit_status,gisobjid,unit_commiss,unit_updated,unit_withdraw
0,-,Fordham Bedford Lot Busters,PARK,M-11,D,100004689.0,2010-01-06,2018-12-20 12:40:52.483,2010-04-01
1,11772,Fordham Bedford Lot Busters,PARK,M-11,D,NaN,2011-07-28,2018-12-20 12:40:52.810,2014-04-11
2,12654,Fordham Bedford Lot Busters,PARK,M-11,D,NaN,2012-03-06,2018-12-20 12:40:52.983,2014-04-11
3,14301,Fordham Bedford Lot Busters,PARK,M-11,D,NaN,2013-06-03,2018-12-20 12:40:53.150,2014-04-11
4,14918,ZONE,ZONE,M-11,D,100008685.0,2015-02-19,2018-12-20 12:41:07.357,2015-02-19


In [53]:
temp_df = before_season_definition.merge(
    before_sla_season[before_sla_season['year_round'] == False],
    on='season_id')[[
        'season_id', 'year_round', 'date_ref_fixed', 'month_name_desc',
        'date_ref_day_number', 'date_type_id'
    ]]
temp_df

,season_id,year_round,date_ref_fixed,month_name_desc,date_ref_day_number,date_type_id
0,2,False,True,May,1.0,1
1,2,False,True,October,1.0,2
2,3,False,True,March,1.0,1
3,3,False,True,October,31.0,2
4,10,False,True,May,1.0,1
5,10,False,True,October,1.0,2
6,9,False,True,March,1.0,1
7,9,False,True,October,31.0,2
8,12,False,True,May,1.0,1
9,12,False,True,October,1.0,2


In [54]:
temp_df = before_season_definition.merge(
    before_sla_season[before_sla_season['year_round'] == False],
    on='season_id')[[
        'season_id',  'month_name_desc',
        'date_type_id'
    ]].merge(month_tbl[['month_name_desc','month_name_num']], on='month_name_desc')
temp_df

,season_id,month_name_desc,date_type_id,month_name_num
0,2,May,1,5
1,10,May,1,5
2,12,May,1,5
3,16,May,1,5
4,18,May,2,5
5,21,May,1,5
6,24,May,1,5
7,2,October,2,10
8,3,October,2,10
9,10,October,2,10


In [55]:
temp_df2 = temp_df.sort_values(by='date_type_id').groupby('season_id').first().merge(
    temp_df.sort_values(by='date_type_id').groupby('season_id').last(),
    left_index=True,
    right_index=True)[['month_name_num_x',
                       'month_name_num_y']].rename(columns={
                           'month_name_num_x': 'start',
                           'month_name_num_y': 'end'
                       })
temp_df2

,start,end
season_id,,
2,5,10
3,3,10
9,3,10
10,5,10
11,3,10
12,5,10
14,4,11
16,5,9
17,4,9


In [56]:
temp_df2 = temp_df.sort_values(by='date_type_id').groupby('season_id').first().merge(
    temp_df.sort_values(by='date_type_id').groupby('season_id').last(),
    left_index=True,
    right_index=True)[['month_name_num_x',
                       'month_name_num_y']].rename(columns={
                           'month_name_num_x': 'start',
                           'month_name_num_y': 'end'
                       })

In [57]:
def get_diff(row, col_strt, col_end):
    if row[col_end]-row[col_strt]>0:
        return row[col_end]-row[col_strt]
    elif row[col_end]-row[col_strt]<0:
        return 12 + row[col_end]-row[col_strt]

In [58]:
temp_df2['diff'] = temp_df2.apply(lambda row: get_diff(row, 'start', 'end'), axis = 1)

In [59]:
range_vals = []
for i in temp_df2['diff'].values:
    if (i < 2) | (i > 10):
        range_vals.append(False)
    else:
        range_vals.append(True)           

In [60]:
# range_vals

In [61]:
results.append(False not in range_vals)

### Eighth before test

In [62]:
timing.append('before')

In [63]:
descriptions.append(
    '''For year-round seasons, test that 'effective_start' in tbl_sla_season_date\
    is equal to 'effective_start' in tbl_sla_season ''')

In [64]:
temp_df = before_season_date.merge(
    before_sla_season[before_sla_season['year_round'] == True],
    on='season_id')[[
        'season_id', 'year_round', 'effective_start_x', 'effective_start_y'
    ]]
temp_df

,season_id,year_round,effective_start_x,effective_start_y
0,1,True,2014-01-01,2014-01-01
1,4,True,2020-03-22,2020-03-22
2,6,True,2020-07-24,2020-07-24
3,20,True,2020-08-09,2020-08-09
4,23,True,2020-08-16,2020-08-16


In [65]:
results.append(
    len(temp_df) == len(temp_df[temp_df['effective_start_x'] ==
                                temp_df['effective_start_y']]))

### Ninth before test

In [66]:
timing.append('before')

In [67]:
descriptions.append(
    '''For periodic seasons, test that the earliest 'effective_start' for each season\
    in tbl_sla_season_date is equal to 'effective_start' in tbl_sla_season ''')

In [68]:
temp_df = before_season_date.merge(
    before_sla_season[before_sla_season['year_round'] == False],
    on='season_id')[[
        'season_id', 'year_round', 'effective_start_x', 'effective_start_y'
    ]].sort_values(by='effective_start_x').groupby('season_id').first()
temp_df

,year_round,effective_start_x,effective_start_y
season_id,,,
2,False,2019-07-01,2019-07-01
3,False,2019-07-01,2019-07-01
9,False,2020-07-26,2020-07-26
10,False,2020-07-26,2020-07-26
11,False,2020-08-09,2020-08-09
12,False,2020-08-07,2020-08-07
14,False,2020-08-13,2020-08-13
16,False,2020-08-15,2020-08-15
17,False,2020-08-16,2020-08-16


In [69]:
results.append(
    len(temp_df) == len(temp_df[temp_df['effective_start_x'] ==
                                temp_df['effective_start_y']]))

### Tenth before test

In [70]:
timing.append('before')

In [71]:
descriptions.append(
    '''For periodic seasons, test that the latest 'effective_end' for each season\
    in tbl_sla_season_date is equal to 'effective_end' in tbl_sla_season OR today+1year (if null) ''')

In [72]:
temp_df = before_season_date.merge(
    before_sla_season[before_sla_season['year_round'] == False],
    on='season_id')[[
        'season_id', 'year_round', 'effective_start_x', 'effective_end_x', 'effective_end_y'
    ]].sort_values(by='effective_start_x').groupby('season_id').last()
temp_df

,year_round,effective_start_x,effective_end_x,effective_end_y
season_id,,,,
2,False,2020-05-01,2020-07-25,2020-07-25
3,False,2020-03-01,2020-07-25,2020-07-25
9,False,2020-07-26,2020-08-08,2020-08-08
10,False,2020-07-26,2020-08-08,2020-08-08
11,False,2021-03-01,2021-09-04,NaT
12,False,2020-08-07,2020-08-15,2020-08-15
14,False,2021-04-15,2021-09-04,NaT
16,False,2021-05-31,2021-09-04,NaT
17,False,2021-04-15,2021-09-04,NaT


In [73]:
two_conditions = []

In [74]:
two_conditions.append(len(temp_df[~pd.isnull(temp_df['effective_end_y'])]) == len(
    temp_df[(~pd.isnull(temp_df['effective_end_y']))
            & (temp_df['effective_end_x'] == temp_df['effective_end_y'])]))

In [75]:
two_conditions.append(
    len(temp_df[pd.isnull(temp_df['effective_end_y'])]) == len(temp_df[
        (pd.isnull(temp_df['effective_end_y']))
        & (temp_df['effective_end_x'] == datetime.datetime(
            today.year + 1, today.month, today.day))]))

In [76]:
results.append(False not in two_conditions)

### Eleventh before test

In [77]:
timing.append('before')

In [78]:
descriptions.append(
    '''Test to make sure there are no gaps in dates in tbl_sla_season_date ''')

In [83]:
sql_gaps_1 = '''
select season_id,
       datediff(day, min(effective_start), max(effective_end)) as daydiff
from sladb.dbo.tbl_sla_season_date
group by season_id
'''

sql_gaps_2 = '''select season_id,
       case when count(*) > 1 then sum(daydiff) + (count(*) - 1)
            else sum(daydiff)
       end as daydiff
from (select season_id,
             datediff(day, effective_start, effective_end) as daydiff
      from sladb.dbo.tbl_sla_season_date) as t
group by season_id
'''

In [84]:
gaps_df1 = pd.read_sql(con = engine, sql = sql_gaps_1)
gaps_df2 = pd.read_sql(con = engine, sql = sql_gaps_2)

In [85]:
temp_df = gaps_df1.merge(gaps_df2, on='season_id')

In [86]:
results.append(len(temp_df)==len(temp_df[temp_df['daydiff_x']==temp_df['daydiff_y']]))

## Make Updates

## Execute all of the sql queries AFTER UPDATES and store the results

## List of tuples with final test results

In [95]:
list(zip(timing, descriptions, results))

[('before',
  "In tbl_unit_change_request, test to make sure the 'effective_start_adj' is between 'effective_start_adj' and 'effective_end_adj' from tbl_sla_season if not null. If 'effective_end_adj' is null, replace with today's date+1year. If this condition is not met, 'sla_change_status' must == 4",
  False),
 ('before',
  "In tbl_unit_sla_season, test that 'effective_start_adj' is the Sunday after 'effective_start' or the Sunday after 'created_date_utc' in tbl_change_request_status",
  True),
 ('before',
  "In tbl_unit_sla_season (or tbl_change_request) test that each unit only has one entry with 'sla_change_status'==1",
  True),
 ('before',
  "In tbl_change_request test that each entry has a unique combination 'unit_id', 'effective_start_adj', and 'sla_change_status' fields",
  True)]

In [96]:
pd.DataFrame(list(zip(timing, descriptions, results)), columns = {'before/after', 'test_desc', 'result'})             

,result,before/after,test_desc
0,before,"In tbl_unit_change_request, test to make sure ...",False
1,before,"In tbl_unit_sla_season, test that 'effective_s...",True
2,before,In tbl_unit_sla_season (or tbl_change_request)...,True
3,before,In tbl_change_request test that each entry has...,True
